In [1]:
from google.cloud import billing_v1
from google.protobuf.json_format import MessageToDict
import pandas as pd

In [2]:
# GET service list
def getServiceList():
    service_request = billing_v1.ListServicesRequest()
    return client.list_services(request=service_request)


In [3]:
# GET skus list
def getSkusList(service_name):
    page_result_list = []
    sku_request = billing_v1.ListSkusRequest(parent=service_name)
    page_result = client.list_skus(request=sku_request)
    page_result_list.append(page_result.skus)

    while(page_result.next_page_token != '') :
        sku_request = billing_v1.ListSkusRequest(parent=service_name, page_token=page_result.next_page_token)
        page_result = client.list_skus(request=sku_request)
        page_result_list.append(page_result.skus)
            
    return page_result_list
    

Main

In [4]:
global client 
client = billing_v1.CloudCatalogClient()

In [5]:
# get service result
service_page_result = getServiceList()

In [6]:
# add service informations to list
service_name_list= []
service_service_id_list = []
service_display_name_list = []
service_business_entity_name_list=[]
for service in service_page_result:
    service_name_list.append(service.name)
    service_service_id_list.append(service.service_id)
    service_display_name_list.append(service.display_name)
    service_business_entity_name_list.append(service.business_entity_name)

# make service lists to dataframe
df_service = pd.DataFrame()
add_df_service = {'name':service_name_list, 'service_id' : service_service_id_list, 'display_name' : service_display_name_list, 'business_entity_name' : service_business_entity_name_list}
for key in add_df_service.keys():
    df_service[key] = add_df_service[key]

In [7]:
# extract service to find its skus
find_service_list= []
find_service_list.append(df_service.loc[df_service['display_name'] == 'Compute Engine'])

In [8]:
# get skus information and add to list
skus_page_result = []
for service in find_service_list:
    skus_page_result = getSkusList(service['name'].values[0])

In [9]:
# make skus list
skus_list = []
for page in skus_page_result:
    for sku in page:
        skus_list.append(sku)

In [10]:
# add sku informaions to list
sku_service_display_name = []
sku_name_list = []
sku_sku_id_list = []
sku_description_list = []
sku_resource_family_list = []
sku_resource_group_list = []
sku_usage_type_list = []
sku_service_regions_list = []
sku_pricing_info_list = []
sku_service_provider_name_list = []

for sku in skus_list:
    sku_service_display_name.append(sku.category.service_display_name)
    sku_name_list.append(sku.name)
    sku_sku_id_list.append(sku.sku_id)
    sku_description_list.append(sku.description)
    sku_resource_family_list.append(sku.category.resource_family)
    sku_resource_group_list.append(sku.category.resource_group)
    sku_usage_type_list.append(sku.category.usage_type)
    sku_service_regions_list.append(sku.service_regions)
    sku_pricing_info_list.append(sku.pricing_info)
    sku_service_provider_name_list.append(sku.service_provider_name)

In [11]:
# make sku dataframe
df_sku = pd.DataFrame()
add_df_sku = {'service_display_name':sku_service_display_name, 
                    'name': sku_name_list,
                    'sku_id' : sku_sku_id_list,
                    'description' : sku_description_list,
                    'resource_family' : sku_resource_family_list,
                    'resource_group' : sku_resource_group_list,
                    'usage_type' : sku_usage_type_list,
                    'service_regions' : sku_service_regions_list,
                    'pricing_info' : sku_pricing_info_list,
                    'service_provicer_name' : sku_service_provider_name_list}

for key in add_df_sku.keys():
    df_sku[key] = add_df_sku[key]